In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import os
from matplotlib.cbook import boxplot_stats
import pickle
sbn.set_context(font_scale=1.5)

In [2]:
#loading the HR data
with open ('./Clean Data R00/HR_DF_TIMECORRECTED.pickle', 'rb') as handle:
    hr = pickle.load(handle)
#Loading a df that has the timestamp. in this case activity    
with open ('./Clean Data R00/ACTIVITY_R00.pickle', 'rb') as handle:
    activity = pickle.load(handle)

In [3]:
hr.head()

,TIME,VALUE,ID,DATE
0,00:00:00,73,1002,0716
1,00:00:15,72,1002,0716
2,00:00:20,70,1002,0716
3,00:00:35,70,1002,0716
4,00:00:40,65,1002,0716


In [4]:
hr.shape

(3773961, 4)

In [5]:
activity.head()

,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories,DATE,ID,Experiment
0,2015-07-16,"4,200","35,439",15.56,53,280,327,166,162,"3,305",0716,1002,R00
1,2015-07-17,"3,789","28,843",12.67,48,347,273,168,127,"2,794",0717,1002,R00
2,2015-07-18,"4,478","39,139",17.24,102,340,400,124,194,"3,629",0718,1002,R00
3,2015-07-19,"4,715","45,343",19.94,116,327,391,121,248,"3,920",0719,1002,R00
4,2015-07-20,"3,948","31,182",13.72,61,407,335,114,157,"3,001",0720,1002,R00


In [10]:
#We do a left join on the df we want to add date
HR_date = hr.merge(activity.loc[:,['Date','DATE','ID', 'Experiment']], how ='left', on = ['ID','DATE']).copy()

In [11]:
HR_date.head()

,TIME,VALUE,ID,DATE,Date,Experiment
0,00:00:00,73,1002,0716,2015-07-16,R00
1,00:00:15,72,1002,0716,2015-07-16,R00
2,00:00:20,70,1002,0716,2015-07-16,R00
3,00:00:35,70,1002,0716,2015-07-16,R00
4,00:00:40,65,1002,0716,2015-07-16,R00


In [12]:
#sanity check
HR_date.shape[0] == hr.shape[0]

True

In [13]:
# Something is wrong. I have missing 
HR_date.isna().sum()

TIME               0
VALUE              0
ID                 0
DATE               0
Date          289709
Experiment    289709
dtype: int64

In [18]:
#Only one subject shows no date fields
HR_date.loc[HR_date['Date'].isna(),:]['ID'].unique()

array(['1085'], dtype=object)

In [19]:
HR_date.loc[HR_date['ID'] == '1085',:]

,TIME,VALUE,ID,DATE,Date,Experiment
3249277,00:00:01,80,1085,1005,NaT,NaN
3249278,00:00:04,80,1085,1005,NaT,NaN
3249279,00:00:07,80,1085,1005,NaT,NaN
3249280,00:00:10,80,1085,1005,NaT,NaN
3249281,00:00:11,79,1085,1005,NaT,NaN
...,...,...,...,...,...,...
3538981,23:58:45,68,1085,1014,NaT,NaN
3538982,23:58:48,68,1085,1014,NaT,NaN
3538983,23:58:51,68,1085,1014,NaT,NaN
3538984,23:58:54,68,1085,1014,NaT,NaN


1085 is missing all the activity values

In [22]:
activity.loc[activity['ID']=='1085',:].head()

,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories,DATE,ID,Experiment
332,2016-10-29,"2,310","13,828",5.40,28,"1,071",182,125,62,"1,366",1029,1085,R00
333,2016-10-30,"2,703","16,628",6.50,21,566,485,87,26,"1,942",1030,1085,R00
334,2016-10-31,"1,846","7,946",3.11,9,"1,177",237,22,4,810,1031,1085,R00
335,2016-11-01,"2,073","11,470",4.48,19,"1,116",266,42,16,"1,103",1101,1085,R00
336,2016-11-02,"2,282","18,177",7.10,48,789,108,93,102,"1,295",1102,1085,R00


In [23]:
hr.loc[hr['ID']=='1085',:].head()

,TIME,VALUE,ID,DATE
3249277,00:00:01,80,1085,1005
3249278,00:00:04,80,1085,1005
3249279,00:00:07,80,1085,1005
3249280,00:00:10,80,1085,1005
3249281,00:00:11,79,1085,1005


In [24]:
#lets import sleep activiy to see if this is also like this there
#Loading a df that has the timestamp. in this case activity    
with open ('./Clean Data R00/SLEEP_R00.pickle', 'rb') as handle:
    sleep = pickle.load(handle)

In [26]:
sleep.loc[sleep['ID'] == '1085',:].head()

,Date,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,DATE,ID,Experiment
332,2016-10-29,0,0,0,0,1029,1085,R00
333,2016-10-30,255,19,1,276,1030,1085,R00
334,2016-10-31,0,0,0,0,1031,1085,R00
335,2016-11-01,0,0,0,0,1101,1085,R00
336,2016-11-02,318,29,2,348,1102,1085,R00


There is a problem with thw subject 1085. The HR `DATE` do not match the sleep date nor the activity `DATE`

In [27]:
#let's see if the same happend with setps
with open ('./Clean Data R00/STEPS_TIMESERIES_R00.pickle', 'rb') as handle:
    steeps = pickle.load(handle)

In [30]:
steeps.isna().sum()

TIME         0
VALUE        0
ID           0
DATE         0
Date     14400
dtype: int64

We have a problem. Let's check if is the subject 1085

In [39]:
#YES it is. We need to figure it out why
steeps.loc[steeps['ID']=='1085']

,TIME,VALUE,ID,DATE,Date
443520,00:00:00,0.0,1085,1005,NaT
443521,00:01:00,49.0,1085,1005,NaT
443522,00:02:00,22.0,1085,1005,NaT
443523,00:03:00,40.0,1085,1005,NaT
443524,00:04:00,27.0,1085,1005,NaT
...,...,...,...,...,...
457915,23:55:00,0.0,1085,1014,NaT
457916,23:56:00,0.0,1085,1014,NaT
457917,23:57:00,0.0,1085,1014,NaT
457918,23:58:00,0.0,1085,1014,NaT


## I figured it out!!
The file `C:\Users\joaqu\Box\Emotional-State-Capstone\data\R00\fitbit\fitbit_detailed\R00_1085_Activity_Tracking` is in reality the subject 1081

In [47]:
pd.Series(hr.loc[hr['ID']=='1081','VALUE'], hr.loc[hr['ID']=='1085','VALUE'])

False

In [48]:
 hr.loc[hr['ID']=='1085','VALUE']

3249277    80
3249278    80
3249279    80
3249280    80
3249281    79
           ..
3538981    68
3538982    68
3538983    68
3538984    68
3538985    68
Name: VALUE, Length: 289709, dtype: int32

In [53]:
sum(hr.loc[hr['ID']=='1081','VALUE'].values == hr.loc[hr['ID']=='1085','VALUE'].values)

289709

They match 100%

## I am going to save the df only because wee need the datetime to run some models. Nevertheless we should be cautious about this and resolve this problem. If it is not possible we should remove subject 1085 when we apply a TS model that use HR or Steps.

In [56]:
with open('./Clean Data R00/HR_with_dates_R00.pickle', 'wb') as handle:
    pickle.dump(HR_date, handle)